In [5]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

In [9]:
# Model 248 is for canceling error
# Model 14/167 for a type three error / echelon
# Model 7 for type 1 error
# Model 18 for type 2 error

######## examine model 628 for large echelon error ###########
simple = load_file_from_curated_data(167)
# for r in simple.reactions:
#   if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
#     print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  [statPhosphorylation:  -> {species_test}]
Type III error:  [PstatDimerisation: 2.00 {Pstat_sol} -> 2.00 {Pstat_nuc=stat_nuc=stat_sol}]
Type I-SOM error:  set()


In [3]:
count = 0
for som in m.nodes:
  if len(som.molecules)>1:
    count = count + len(som.reactions)
print(count)

14


In [4]:
np.linalg.cond(m.lower_inverse)

1.6403882032022075

In [13]:
len(simple.reactions)

827

In [16]:
resulting_series = gr.getResultingSeries("statPhosphorylation")

In [11]:
len(simple.reactions)

827

In [12]:
type1_report, error_num = gr.reportTypeOneError([m.type_one_errors[1]], explain_details=True)
print(type1_report)
print(error_num)

We detected a mass imbalance from the following reactions:


c154 = c86 by reaction(s):
1. v537: c154 -> c86

c86 = c160 by reaction(s):
2. v601: c160 -> c86

However, c154 < c160 by reaction(s):
3. v13: c10 + c154 -> c160


----------------------------------------------------------------------



**********************************************************************


[3]


In [4]:
[m.type_three_errors[0]]

[Reaction29: {K_MAPKP_1_2} -> {MAPKP}]

In [9]:
a = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
a

In [10]:
type3_report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
print(type3_report)
print(error_num)

We detected a mass imbalance from the following reactions:

1. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
2. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc

----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The molecules within a curly bracket have the same atomic mass.)

3. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc;   {PstatDimer_sol=PstatDimer_nuc}
4. stat_export: stat_sol -> stat_nuc;   {stat_sol=stat_nuc}
5. statDephosphorylation: Pstat_nuc -> stat_nuc;   {stat_sol=stat_nuc=Pstat_nuc}

----------------------------------------------------------------------

These multi-uni (uni-multi) reactions created mass-inequality.

6. statPhosphorylation: stat_sol -> Pstat_sol + species_test

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.

(pseudo 1.) PstatDimerisation: 2.00 {Pstat_sol} -> {PstatDime

In [22]:
type2_report, error_num = gr.reportTypeTwoError([m.type_two_errors[1]], explain_details=True)
print(type2_report)
print(error_num)

We detected a mass imbalance from the following reactions:

These uni-uni reactions created mass-equivalence.

1. FFH2syn: FH2f -> FFH2
2. SHMTr: CH2FH4 -> FH4
3. DHFReductase: FH2f -> FH4

----------------------------------------------------------------------

The following reactions create mass-inequality.

4. MTHFR: CH2FH4 + NADPH -> CH3FH4
5. MTR: CH3FH4 + homocysteine -> FH4

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.

(pseudo 4.) MTHFR: {CH2FH4=FFH2=FH2f=FH4} + {NADPH} -> {CH3FH4}
(pseudo 5.) MTR: {homocysteine} + {CH3FH4} -> {CH2FH4=FFH2=FH2f=FH4}

----------------------------------------------------------------------

However, the above pseudo reactions imply the following inequalities:

{CH2FH4=FFH2=FH2f=FH4} < {CH3FH4} < {CH2FH4=FFH2=FH2f=FH4}

This indicates a mass conflict between reactions.

----------------------------------------------------------------------

-----------

In [12]:
type2_report

'We detected a mass imbalance from the following reactions:\n\nThese uni-uni reactions created mass-equivalence.\n\n1. FFH2syn: FH2f -> FFH2\n2. SHMTr: CH2FH4 -> FH4\n3. DHFReductase: FH2f -> FH4\n\n----------------------------------------------------------------------\n\nThe following reactions create mass-inequality.\n\n4. HCOOHtoCHOFH4: FH4 + formate + ATP -> CHOFH4\n5. MTHFD: CH2FH4 + NADP -> CHOFH4\n6. ATIC7: CHOFH4 + AICAR -> FH4\n\n----------------------------------------------------------------------\n\nBased on the reactions above, we have mass-equivalent pseudo reactions.\n\n(pseudo 4.) HCOOHtoCHOFH4: {CH2FH4=FFH2=FH2f=FH4} + {formate} + {ATP} -> {CHOFH4}\n(pseudo 5.) MTHFD: {CH2FH4=FFH2=FH2f=FH4} + {NADP} -> {CHOFH4}\n(pseudo 6.) ATIC7: {CHOFH4} + {AICAR=FGAR} -> {CH2FH4=FFH2=FH2f=FH4}\n\n----------------------------------------------------------------------\n\nHowever, the above pseudo reactions imply the following inequalities:\n\n{CH2FH4=FFH2=FH2f=FH4} < {CHOFH4} < {CH2FH

In [11]:
echelon_report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
print(echelon_report)
print(error_num)


We detected a mass imbalance
:  -> species_test

from the following reactions.

1. statPhosphorylation: stat_sol -> Pstat_sol + species_test
2. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
3. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc

----------------------------------------------------------------------

These uni-uni reactions created mass-equivalence.
(The molecules within a curly bracket have the same atomic mass.)

4. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc;   {PstatDimer_sol=PstatDimer_nuc}
5. stat_export: stat_sol -> stat_nuc;   {stat_sol=stat_nuc}
6. statDephosphorylation: Pstat_nuc -> stat_nuc;   {stat_sol=stat_nuc=Pstat_nuc}

----------------------------------------------------------------------

Based on the reactions above, we have mass-equivalent pseudo reactions.
(pseudo 1.) statPhosphorylation: {Pstat_nuc=stat_nuc=stat_sol} -> {species_test} + {Pstat_sol}
(pseudo 2.) PstatDimerisation: 2.00 {Pstat_sol} -> {PstatDimer_nuc=PstatDimer_sol}


In [15]:
echelon_report

False

In [3]:
canceling_report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
print(canceling_report)
print(error_num)

We detected a mass imbalance from the following reactions:

1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP

2. ATPase: ATP -> ADP

*ATP and ADP have the same mass according to the above reaction


Therefore, they will result in empty product with zero mass:

OxidativePhosphorylation: CTtis -> 

This indicates a mass conflict between reactions.

----------------------------------------------------------------------

----------------------------------------------------------------------



**********************************************************************


[2]


In [7]:
data_dir=cn.BIOMODELS_DIR
#files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
files = [f for f in os.listdir(data_dir) if f[:7] == "BIOMD00"]
paths = [os.path.join(data_dir, filename) for filename in files]
# this will test which model has which error;;
model_count = 0
count = 0
simple = SimpleSBML
for file in files:
  try:
    simple.initialize(os.path.join(data_dir, file))
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    if m.analyze(error_details=False):
      if (len(simple.reactions) > 100) and (len(m.type_one_errors)>1):
        print(file, "has type I error with more than 100 reactions")
        count += 1
    if count > 5:
      break
  except:
    pass

we are doing model 100
we are doing model 200
BIOMD0000000255_url.xml has type I error with more than 100 reactions
BIOMD0000000244_url.xml has type I error with more than 100 reactions
BIOMD0000000560_url.xml has type I error with more than 100 reactions
we are doing model 300
BIOMD0000000293_url.xml has type I error with more than 100 reactions
BIOMD0000000588_url.xml has type I error with more than 100 reactions
BIOMD0000000223_url.xml has type I error with more than 100 reactions


In [19]:
# this will test which model has which error;;
model_count = 0
count = 0
for i in range(1, 150):
  try:
    simple = load_file_from_curated_data(i)
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    m.analyze(error_details=False)
    if m.echelon_errors or m.type_three_errors:
        print("model", model_count, " has an echelon or type three error!")
        if count == 5:
          break
  except:
    pass

model 14  has an echelon or type three error!
model 49  has an echelon or type three error!
model 51  has an echelon or type three error!
model 52  has an echelon or type three error!
we are doing model 100
model 102  has an echelon or type three error!
model 109  has an echelon or type three error!
model 140  has an echelon or type three error!
